<h1 style='font-size:40px'> Machine Learning with MLlib</h1>
<div> 
    <ul style='font-size:20px'> 
        <li> 
            Iremos aprender a utilizar a spark.ml com um dataset sobre aluguéis do Airbnb em São Francisco.  
        </li>
    </ul>
</div>

<h2 style='font-size:30px'> Data Ingestion and Exploration</h2>

In [2]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/databricks/LearningSparkV2/master/databricks-datasets/learning-spark-v2/sf-airbnb/sf-airbnb.csv',
                usecols=['neighbourhood', 'room_type', 'bedrooms', 'bathrooms', 'number_of_reviews', 'price'])

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('C.10').getOrCreate()

22/06/14 12:49:51 WARN Utils: Your hostname, veiga-Inspiron resolves to a loopback address: 127.0.1.1; using 192.168.15.21 instead (on interface wlp7s0)
22/06/14 12:49:51 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/06/14 12:49:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [59]:
airbnbDF = spark.createDataFrame(df, schema='''`neighborhood` STRING, `room_type` STRING, `bathrooms` FLOAT, `bedrooms` FLOAT, `price` STRING, 
                            `number_of_reviews` INT''')

In [62]:
from pyspark.ml.feature import Imputer
import pyspark.sql.functions as F

num_cols = ['bathrooms', 'bedrooms', 'price', 'number_of_reviews']
imputer = Imputer(strategy='median', inputCols=num_cols, outputCols=[f'{col}_na' for col in num_cols])
airbnbDF = (airbnbDF.withColumn('price', F.substring('price', 2, 10))
        .withColumn('price',F.col('price').cast('float'))
        .where('price>0'))

airbnbDF = imputer.fit(airbnbDF).transform(airbnbDF)
airbnbDF = airbnbDF.drop('bathrooms', 'bedrooms', 'price', 'number_of_reviews')
airbnbDF.show()

+--------------------+---------------+------------+-----------+--------+--------------------+
|        neighborhood|      room_type|bathrooms_na|bedrooms_na|price_na|number_of_reviews_na|
+--------------------+---------------+------------+-----------+--------+--------------------+
|      Bernal Heights|Entire home/apt|         1.0|        2.0|     5.0|                 111|
|Western Addition/...|Entire home/apt|         1.5|        2.0|     5.0|                  27|
|Western Addition/...|Entire home/apt|         1.0|        2.0|     5.0|                  31|
|    Mission District|   Private room|         1.0|        1.0|     9.0|                 647|
|        Potrero Hill|   Private room|         1.0|        1.0|     5.0|                 453|
|    Mission District|Entire home/apt|         1.0|        2.0|     5.0|                 320|
|        Lower Haight|Entire home/apt|         1.0|        3.0|     7.0|                  37|
|        Hayes Valley|Entire home/apt|         2.0|        3

<p style='color:red'> Creating Training and Test Data Sets